![cyber_photo](cyber_photo.jpg)

Cyber threats are a growing concern for organizations worldwide. These threats take many forms, including malware, phishing, and denial-of-service (DOS) attacks, compromising sensitive information and disrupting operations. The increasing sophistication and frequency of these attacks make it imperative for organizations to adopt advanced security measures. Traditional threat detection methods often fall short due to their inability to adapt to new and evolving threats. This is where deep learning models come into play.

Deep learning models can analyze vast amounts of data and identify patterns that may not be immediately obvious to human analysts. By leveraging these models, organizations can proactively detect and mitigate cyber threats, safeguarding their sensitive information and ensuring operational continuity.

As a cybersecurity analyst, you identify and mitigate these threats. In this project, you will design and implement a deep learning model to detect cyber threats. The BETH dataset simulates real-world logs, providing a rich source of information for training and testing your model. The data has already undergone preprocessing, and we have a target label, `sus_label`, indicating whether an event is malicious (1) or benign (0).

By successfully developing this model, you will contribute to enhancing cybersecurity measures and protecting organizations from potentially devastating cyber attacks.


### The Data

| Column     | Description              |
|------------|--------------------------|
|`processId`|The unique identifier for the process that generated the event - int64 |
|`threadId`|ID for the thread spawning the log - int64|
|`parentProcessId`|Label for the process spawning this log - int64|
|`userId`|ID of user spawning the log|Numerical - int64|
|`mountNamespace`|Mounting restrictions the process log works within - int64|
|`argsNum`|Number of arguments passed to the event - int64|
|`returnValue`|Value returned from the event log (usually 0) - int64|
|`sus_label`|Binary label as suspicous event (1 is suspicious, 0 is not) - int64|

More information on the dataset: [BETH dataset](accreditation.md)

In [12]:
# ===============================
# 1. Import libraries
# ===============================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchmetrics
import pandas as pd

In [13]:
# ===============================
# 2. Load preprocessed CSV files
# ===============================
train_df = pd.read_csv('labelled_train.csv')
val_df   = pd.read_csv('labelled_validation.csv')
test_df  = pd.read_csv('labelled_test.csv')

In [14]:
# ===============================
# 3. Split features and labels
# ===============================
X_train = train_df.drop(columns=['sus_label']).values
y_train = train_df['sus_label'].values

X_val = val_df.drop(columns=['sus_label']).values
y_val = val_df['sus_label'].values

In [15]:
# ===============================
# 4. Convert to PyTorch tensors
# ===============================
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

X_val   = torch.tensor(X_val,   dtype=torch.float32)
y_val   = torch.tensor(y_val,   dtype=torch.long)

In [16]:
# ===============================
# 5. Create TensorDataset + DataLoader
# ===============================
train_ds = TensorDataset(X_train, y_train)
val_ds   = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False)

In [17]:
# ===============================
# 6. Define Model
# ===============================
input_dim = X_train.shape[1]   # number of columns

model = nn.Sequential(
    nn.Linear(input_dim, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 2)   # binary classification → 2 outputs
)

In [18]:
# ===============================
# 7. Loss + Optimizer
# ===============================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [19]:
# ===============================
# 8. Accuracy Metric
# ===============================
accuracy_metric = torchmetrics.Accuracy(task="multiclass", num_classes=2)

In [20]:
# ===============================
# 9. Training Loop (10 epochs)
# ===============================
num_epochs = 10

for epoch in range(num_epochs):

    # ---- Training ----
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # ---- Validation ----
    model.eval()
    accuracy_metric.reset()

    with torch.no_grad():
        for features, labels in val_loader:
            outputs = model(features)
            preds = outputs.argmax(dim=-1)
            accuracy_metric.update(preds, labels)

    val_accuracy = accuracy_metric.compute().item()

    print(f"Epoch {epoch+1}/{num_epochs} | Validation Accuracy = {val_accuracy:.4f}")

Epoch 1/10 | Validation Accuracy = 0.9958
Epoch 2/10 | Validation Accuracy = 0.9958
Epoch 3/10 | Validation Accuracy = 0.9958
Epoch 4/10 | Validation Accuracy = 0.9958
Epoch 5/10 | Validation Accuracy = 0.9958
Epoch 6/10 | Validation Accuracy = 0.9958
Epoch 7/10 | Validation Accuracy = 0.9958
Epoch 8/10 | Validation Accuracy = 0.9958
Epoch 9/10 | Validation Accuracy = 0.9958
Epoch 10/10 | Validation Accuracy = 0.9958


In [21]:
# ===============================
# Final required output variable
# (The project needs val_accuracy defined)
# ===============================
val_accuracy = float(val_accuracy)
val_accuracy

0.9958405494689941